In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

In [10]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data with timestamp parsed as datetime
    """
    # YOUR CODE HERE
    # Load the CSV file using pandas
    # Make sure to parse the timestamp column as datetime
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    
    return df # Replace with actual implementation

df  = load_data('data/synthetic_health_data.csv')
print(df)

      patient_id                  timestamp  age  systolic_bp  diastolic_bp  \
0              1 2023-01-29 00:00:00.000000   57   113.063416     84.069561   
1              1 2023-01-31 07:33:55.507789   57   121.598849     89.672279   
2              1 2023-02-02 00:15:11.379377   57   126.623222     87.619685   
3              1 2023-02-04 09:37:12.589164   57   136.999366     89.199774   
4              1 2023-02-04 20:56:52.838198   57   127.546919     92.644673   
...          ...                        ...  ...          ...           ...   
7321         150 2023-03-18 09:08:49.029823   54   115.038254     79.241741   
7322         150 2023-03-20 14:38:22.129593   54   116.389186     70.464818   
7323         150 2023-03-23 09:26:04.210673   54   123.419606     88.213054   
7324         150 2023-03-27 14:17:19.255961   54          NaN     69.539940   
7325         150 2023-04-12 04:12:38.529880   54          NaN     76.992331   

      glucose_level        bmi smoker_status  heart

In [11]:
def extract_rolling_features(df, window_size_seconds):
    """
    Calculate rolling mean and standard deviation for heart rate.
    
    Args:
        df: DataFrame with timestamp and heart_rate columns
        window_size_seconds: Size of the rolling window in seconds
        
    Returns:
        DataFrame with added hr_rolling_mean and hr_rolling_std columns
    """
    # YOUR CODE HERE
    # 1. Sort data by timestamp
    df_sorted = df.sort_values('timestamp')
    
    # 2. Set timestamp as index (this allows time-based operations)
    df_indexed = df_sorted.set_index('timestamp')
    
    # 3. Calculate rolling mean and standard deviation
    #    - First, create a rolling window object based on time:
    rolling_window = df_indexed['heart_rate'].rolling(window=f'{window_size_seconds}s')
    #    - Then calculate statistics on this window:
    hr_mean = rolling_window.mean()
    hr_std = rolling_window.std()
    
    # 4. Add the new columns back to the dataframe
    df_indexed['hr_rolling_mean'] = hr_mean
    df_indexed['hr_rolling_std'] = hr_std
    
    # 5. Reset index to bring timestamp back as a column
    df_result = df_indexed.reset_index()
    
    # 6. Handle any NaN values (rolling calculations create NaNs at the beginning)
    #    - You can use fillna, dropna, or other methods depending on your strategy
    df_result = df_result.fillna(method='bfill')  # Example: backward fill
    
    # Placeholder return - replace with your implementation
    return df_result

roll_df = extract_rolling_features(df, 1)
print(roll_df)

                      timestamp  patient_id  age  systolic_bp  diastolic_bp  \
0    2023-01-01 00:00:00.000000           9   42   109.271202     84.872277   
1    2023-01-01 00:00:00.000000          72   73   117.499246     80.624776   
2    2023-01-01 00:00:00.000000         131   26   125.396859     75.090352   
3    2023-01-01 00:00:00.000000         116   54   124.532627     78.492678   
4    2023-01-02 00:00:00.000000          56   63   111.304514     67.796714   
...                         ...         ...  ...          ...           ...   
7321 2023-07-20 21:01:11.701224         109   53   130.694575     74.261794   
7322 2023-07-23 01:45:47.772214           5   46   108.477241     88.742732   
7323 2023-07-23 06:55:40.341291          16   41   124.742455     83.196582   
7324 2023-08-03 10:54:11.976978         109   53   130.912449     69.467888   
7325 2023-08-05 13:16:36.634428          90   57   133.101666     82.658927   

      glucose_level        bmi smoker_status  heart

/tmp/ipykernel_15464/2178216291.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_result = df_result.fillna(method='bfill')  # Example: backward fill


In [12]:
def prepare_data_part2(df_with_features, test_size=0.2, random_state=42):
    """
    Prepare data for modeling with time-series features.
    
    Args:
        df_with_features: DataFrame with original and rolling features
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Select relevant features including the rolling features
    # 2. Select target variable (disease_outcome)
    # 3. Split data into training and testing sets
    # 4. Handle missing values
    
    feature_cols = [
        'age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi',
        'heart_rate', 'hr_rolling_mean', 'hr_rolling_std'
    ]
    X = df_with_features[feature_cols]
    y = df_with_features['disease_outcome']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    return X_train, X_test, y_train, y_test

In [13]:
def train_random_forest(X_train, y_train, n_estimators=100, max_depth=10, random_state=42):
    """
    Train a Random Forest classifier.
    
    Args:
        X_train: Training features
        y_train: Training target
        n_estimators: Number of trees in the forest
        max_depth: Maximum depth of the trees
        random_state: Random seed for reproducibility
        
    Returns:
        Trained Random Forest model
    """
    # YOUR CODE HERE
    # Initialize and train a RandomForestClassifier
    
    rf = RandomForestClassifier(n_estimators=100, 
                                max_depth=10, 
                                random_state=42)
    
    model = rf.fit(X_train, y_train)
    
    return model  # Replace with actual implementation

In [14]:
def train_xgboost(X_train, y_train, n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42):
    """
    Train an XGBoost classifier.
    
    Args:
        X_train: Training features
        y_train: Training target
        n_estimators: Number of boosting rounds
        learning_rate: Boosting learning rate
        max_depth: Maximum depth of a tree
        random_state: Random seed for reproducibility
        
    Returns:
        Trained XGBoost model
    """
    # YOUR CODE HERE
    # Initialize and train an XGBClassifier
    
    xmod = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    model = xmod.fit(X_train, y_train)
    
    return model  # Replace with actual implementation

In [15]:
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Extract rolling features
    window_size = 300  # 5 minutes in seconds
    df_with_features = extract_rolling_features(df, window_size)
    
    # 3. Prepare data
    X_train, X_test, y_train, y_test = prepare_data_part2(df_with_features)
    
    # 4. Train models
    rf_model = train_random_forest(X_train, y_train)
    xgb_model = train_xgboost(X_train, y_train)
    
    # 5. Calculate AUC scores
    rf_probs = rf_model.predict_proba(X_test)[:, 1]
    xgb_probs = xgb_model.predict_proba(X_test)[:, 1]
    
    rf_auc = roc_auc_score(y_test, rf_probs)
    xgb_auc = roc_auc_score(y_test, xgb_probs)
    
    print(f"Random Forest AUC: {rf_auc:.4f}")
    print(f"XGBoost AUC: {xgb_auc:.4f}")
    
    # 6. Save results
    # (Your code for saving results)
    
    os.makedirs('results', exist_ok=True) # create dir if does not exist

    output_file = 'results/results_part2.txt'

    with open(output_file, "w") as file: # open file for writing
        file.write(f"Random Forest AUC: {rf_auc:.4f}\n")
        file.write(f"XGBoost AUC: {xgb_auc:.4f}\n")

    file.close()

    print(f"\nSynthetic data saved to {output_file}")

/tmp/ipykernel_15464/2178216291.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_result = df_result.fillna(method='bfill')  # Example: backward fill


Random Forest AUC: 0.9743
XGBoost AUC: 0.9965

Synthetic data saved to results/results_part2.txt
